# **A C-LSTM Neural Network for Text Classification**

Convolutional neural network (CNN)
and recurrent neural network (RNN) are two
mainstream architectures for such modeling
tasks, which adopt totally different ways of
understanding natural languages. 

In this work,
we combine the strengths of both architectures
and propose a novel and unified model called
C-LSTM for sentence representation and text
classification. 

C-LSTM utilizes CNN to extract a sequence of higher-level phrase representations, and are fed into a long short-term
memory recurrent neural network (LSTM) to
obtain the sentence representation. 

C-LSTM
is able to capture both local features of phrases
as well as global and temporal sentence semantics. We evaluate the proposed architecture on sentiment classification task.

# Reference

Zhou et al, A C-LSTM Neural Network for Text Classification, arXiv:1511.08630 [cs.CL]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
import keras
from keras.models import Sequential
import keras.layers
from keras.layers import Dense, Embedding, LSTM, GRU, Input, Reshape, Concatenate, Permute, Activation, multiply, Lambda, Conv2D, Bidirectional, Flatten
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras import Model
import keras.utils.np_utils
import tensorflow as tf
from keras import Model
import tensorflow.keras.backend as K

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/IISc_Assignment/DLNLP_A4/TrainData.csv")

In [ ]:
data.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [ ]:
data['Text'][0]

'worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by worldcom. mr ebb

In [ ]:
len(data)

1490

In [ ]:
category = list(data['Category'])

In [ ]:
np.unique(category)

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype='<U13')

In [ ]:
d = {'business' : 0, 'entertainment' : 1, 'politics' : 2, 'sport' : 3, 'tech' : 4}

In [53]:
labels = []

for c in category:
  labels.append(d[c])

In [54]:
train_target = np.array(labels)

In [ ]:
# train_target = keras.utils.np_utils.to_categorical(labels)
# train_target.shape

(1490, 5)

In [55]:
punctuations = '''!()-[]{};:'"\,<>./?@#%^&*_~0123456789'''

pre_data = []
for sentence in data['Text']:
  for ele in sentence:
    if (ele in punctuations):
        sentence = sentence.replace(ele, "")
  pre_data.append(sentence)

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [56]:
lematize_data = []

lemmatizer = WordNetLemmatizer()

for s in pre_data:
  l = []
  for w in s.split():
    w = lemmatizer.lemmatize(w)
    l.append(w)
  
  # pre_data.append(l)
  lematize_data.append(" ".join([i for i in l]))

In [ ]:
lematize_data[0]

'worldcom exboss launch defence lawyer defending former worldcom chief bernie ebbers against a battery of fraud charge have called a company whistleblower a their first witness cynthia cooper worldcom s exhead of internal accounting alerted director to irregular accounting practice at the u telecom giant in her warning led to the collapse of the firm following the discovery of an $bn £bn accounting fraud mr ebbers ha pleaded not guilty to charge of fraud and conspiracy prosecution lawyer have argued that mr ebbers orchestrated a series of accounting trick at worldcom ordering employee to hide expense and inflate revenue to meet wall street earnings estimate but m cooper who now run her own consulting business told a jury in new york on wednesday that external auditor arthur andersen had approved worldcom s accounting in early and she said andersen had given a green light to the procedure and practice used by worldcom mr ebber s lawyer have said he wa unaware of the fraud arguing that a

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [57]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

final_data = []
for sentence in lematize_data:
    final_data.append(" ".join([words for words in sentence.split() if not (words in stop_words or len(words) <=2)]))

In [ ]:
final_data[0]

'worldcom exboss launch defence lawyer defending former worldcom chief bernie ebbers battery fraud charge called company whistleblower first witness cynthia cooper worldcom exhead internal accounting alerted director irregular accounting practice telecom giant warning led collapse firm following discovery $bn £bn accounting fraud ebbers pleaded guilty charge fraud conspiracy prosecution lawyer argued ebbers orchestrated series accounting trick worldcom ordering employee hide expense inflate revenue meet wall street earnings estimate cooper run consulting business told jury new york wednesday external auditor arthur andersen approved worldcom accounting early said andersen given green light procedure practice used worldcom ebber lawyer said unaware fraud arguing auditor alert problem cooper also said shareholder meeting ebbers often passed technical question company finance chief giving brief answer prosecution star witness former worldcom financial chief scott sullivan said ebbers orde

In [58]:
max_len = max([len(s.split()) for s in final_data])

In [ ]:
max_len

1604

In [ ]:
len_list = [len(s.split()) for s in final_data]

In [59]:
max_len = 500

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(final_data)
sequences = tokenizer.texts_to_sequences(final_data)

#pad sequences
train_word_index = tokenizer.word_index
print("number of unique tokens = ", len(train_word_index))



train_padded_sequences = pad_sequences(sequences, padding='post', maxlen = max_len)

number of unique tokens =  22479


In [ ]:
train_padded_sequences[0]

In [ ]:
type(train_word_index)

dict

In [ ]:
from gensim.models import Word2Vec, KeyedVectors
w2v_model = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/IISc_Assignment/DLNLP_Assignment3/GoogleNews-vectors-negative300.bin", binary=True)

In [ ]:
unique_words = len(train_word_index) + 1
w2v_embedding_dim = 300
w2v_embedding = np.zeros((unique_words, w2v_embedding_dim))
count = 0
for word, i in train_word_index.items():
  # if i > num_words:
  #   continue
  if word in w2v_model.vocab:
    embedding = w2v_model.wv[word]
  else:
    embedding = np.random.uniform(-0.25, 0.25, w2v_embedding_dim)
    count += 1
  

  if embedding is not None:
    w2v_embedding[i] = embedding

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


# **CNN-LSTM Model**

In [ ]:
classes = 5
filters = 16
k=5


model_input = Input(shape = (max_len,))

Embedding(unique_words, output_dim = w2v_embedding_dim, weights = [w2v_embedding], trainable = False)

embedding = Embedding(unique_words, output_dim = w2v_embedding_dim, weights = [w2v_embedding], trainable = False)(model_input)

embedding_resize = tf.expand_dims(embedding, -1)

conv_out = Conv2D(filters, (k, w2v_embedding_dim), activation="relu")(embedding_resize)

conv_out = tf.squeeze(conv_out, [2])

# conv_out = conv_out[: , :max_len-k+1, : ]

lstm_out = Bidirectional(LSTM(units= w2v_embedding_dim, dropout=0.2))(conv_out)

model_out = Dense(classes )(lstm_out)

cnn_lstm_model = Model(model_input, model_out)

In [ ]:
cnn_lstm_model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 500)]             0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 500, 300)          6744000   
_________________________________________________________________
tf.expand_dims_7 (TFOpLambda (None, 500, 300, 1)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 496, 1, 16)        24016     
_________________________________________________________________
tf.compat.v1.squeeze_7 (TFOp (None, 496, 16)           0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 600)               760800    
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 3005

In [ ]:
cnn_lstm_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [ ]:
cnn_lstm_model.fit(train_padded_sequences, train_target, batch_size=64, epochs=10, verbose=1, validation_split=0.2)

Epoch 1/10
19/19 [==============================] - 11s 376ms/step - loss: 1.4782 - accuracy: 0.3666 - val_loss: 1.0712 - val_accuracy: 0.6342
Epoch 2/10
19/19 [==============================] - 6s 325ms/step - loss: 0.9894 - accuracy: 0.5747 - val_loss: 0.7729 - val_accuracy: 0.6611
Epoch 3/10
19/19 [==============================] - 6s 325ms/step - loss: 0.6446 - accuracy: 0.7584 - val_loss: 0.5262 - val_accuracy: 0.8356
Epoch 4/10
19/19 [==============================] - 6s 327ms/step - loss: 0.5041 - accuracy: 0.8515 - val_loss: 0.9278 - val_accuracy: 0.6107
Epoch 5/10
19/19 [==============================] - 6s 326ms/step - loss: 0.5219 - accuracy: 0.8146 - val_loss: 0.4736 - val_accuracy: 0.8624
Epoch 6/10
19/19 [==============================] - 6s 327ms/step - loss: 1.0788 - accuracy: 0.6191 - val_loss: 0.8941 - val_accuracy: 0.6611
Epoch 7/10
19/19 [==============================] - 6s 325ms/step - loss: 0.6821 - accuracy: 0.7743 - val_loss: 0.4796 - val_accuracy: 0.8322
Epoch

# **CNN-LSTM Model with attention**

In [ ]:
from keras.layers.core import RepeatVector
classes = 5
filters = 16
k=5


model_input = Input(shape = (max_len,))

Embedding(unique_words, output_dim = w2v_embedding_dim, weights = [w2v_embedding], trainable = False)

embedding = Embedding(unique_words, output_dim = w2v_embedding_dim, weights = [w2v_embedding], trainable = False)(model_input)

embedding_resize = tf.expand_dims(embedding, -1)

conv_out = Conv2D(filters, (k, w2v_embedding_dim), activation="relu")(embedding_resize)

conv_out = tf.squeeze(conv_out, [2])

# conv_out = conv_out[: , :max_len-k+1, : ]

lstm_out = Bidirectional(LSTM(units= w2v_embedding_dim, dropout=0.2, return_sequences = True))(conv_out)

attention = Dense(1)(lstm_out)

attention = Flatten()(attention)

attention = RepeatVector(w2v_embedding_dim * 2)(attention)

attention = Permute([2,1])(attention)

attention_embedding = keras.layers.Multiply()([lstm_out, attention])

attention_embedding = keras.layers.Lambda(lambda xin: K.sum(xin, axis = -2), output_shape = (w2v_embedding_dim * 2,) )(attention_embedding)

model_out = Dense(classes )(attention_embedding)

cnn_lstm_attention_model = Model(model_input, model_out)

In [ ]:
cnn_lstm_attention_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 500)]        0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 500, 300)     6744000     input_5[0][0]                    
__________________________________________________________________________________________________
tf.expand_dims_4 (TFOpLambda)   (None, 500, 300, 1)  0           embedding_9[0][0]                
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 496, 1, 16)   24016       tf.expand_dims_4[0][0]           
____________________________________________________________________________________________

In [ ]:
cnn_lstm_attention_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [ ]:
cnn_lstm_attention_model.fit(train_padded_sequences, train_target, batch_size=64, epochs=20, verbose=1, validation_split=0.2)

Epoch 1/20
19/19 [==============================] - 44s 412ms/step - loss: 1.1475 - accuracy: 0.4706 - val_loss: 0.6221 - val_accuracy: 0.8591
Epoch 2/20
19/19 [==============================] - 6s 341ms/step - loss: 0.3146 - accuracy: 0.9094 - val_loss: 0.2849 - val_accuracy: 0.9228
Epoch 3/20
19/19 [==============================] - 6s 343ms/step - loss: 0.1895 - accuracy: 0.9438 - val_loss: 0.1546 - val_accuracy: 0.9631
Epoch 4/20
19/19 [==============================] - 6s 343ms/step - loss: 0.1186 - accuracy: 0.9622 - val_loss: 0.1670 - val_accuracy: 0.9497
Epoch 5/20
19/19 [==============================] - 6s 341ms/step - loss: 0.1046 - accuracy: 0.9715 - val_loss: 0.1315 - val_accuracy: 0.9530
Epoch 6/20
19/19 [==============================] - 6s 341ms/step - loss: 0.0715 - accuracy: 0.9799 - val_loss: 0.1760 - val_accuracy: 0.9497
Epoch 7/20
19/19 [==============================] - 6s 341ms/step - loss: 0.0775 - accuracy: 0.9782 - val_loss: 0.1388 - val_accuracy: 0.9597
Epoch

# **Loading test data**

In [60]:
test = pd.read_csv("/content/TestData_Inputs.csv")

In [61]:
test.head()

,ArticleId,Text
0,1018,qpr keeper day heads for preston queens park r...
1,1319,software watching while you work software that...
2,1138,d arcy injury adds to ireland woe gordon d arc...
3,459,india s reliance family feud heats up the ongo...
4,1020,boro suffer morrison injury blow middlesbrough...


In [62]:
punctuations = '''!()-[]{};:'"\,<>./?@#%^&*_~0123456789'''

pre_data = []
for sentence in test['Text']:
  for ele in sentence:
    if (ele in punctuations):
        sentence = sentence.replace(ele, "")
  pre_data.append(sentence)

In [63]:
lematize_data = []

lemmatizer = WordNetLemmatizer()

for s in pre_data:
  l = []
  for w in s.split():
    w = lemmatizer.lemmatize(w)
    l.append(w)
  
  # pre_data.append(l)
  lematize_data.append(" ".join([i for i in l]))

In [64]:
final_data = []
for sentence in lematize_data:
    final_data.append(" ".join([words for words in sentence.split() if not (words in stop_words or len(words) <=2)]))

In [65]:
sequences = tokenizer.texts_to_sequences(final_data)
test_padded_sequences = pad_sequences(sequences, padding='post', maxlen = max_len)

In [66]:
test_label = pd.read_excel("Assignment4_TestLabels.xlsx")

In [67]:
labels = test_label['Label - (business, tech, politics, sport, entertainment)']

test_labels = []

for c in labels:
  test_labels.append(d[c])

test_labels = np.array(test_labels)

In [68]:
prediction = cnn_lstm_attention_model.predict(test_padded_sequences)
prediction = np.argmax(prediction, axis = 1)

In [69]:
from sklearn.metrics import f1_score
f1_score(test_labels,prediction,average='micro')

0.9673469387755103

In [70]:
cnn_lstm_attention_model.evaluate(test_padded_sequences, test_labels)

23/23 [==============================] - 2s 102ms/step - loss: 0.2583 - accuracy: 0.9673


[0.25831368565559387, 0.9673469662666321]